![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column                   | Description                                                                      |
|------------------------- |--------------------------------------------------------------------------------- |
| `student_id`             | A unique ID for each student.                                                    |
| `city`                   | A code for the city the student lives in.                                        |
| `city_development_index` | A scaled development index for the city.                                         |
| `gender`                 | The student's gender.                                                            |
| `relevant_experience`    | An indicator of the student's work relevant experience.                          |
| `enrolled_university`    | The type of university course enrolled in (if any).                              |
| `education_level`        | The student's education level.                                                   |
| `major_discipline`       | The educational discipline of the student.                                       |
| `experience`             | The student's total work experience (in years).                                  |
| `company_size`           | The number of employees at the student's current employer.                       |
| `company_type`           | The type of company employing the student.                                       |
| `last_new_job`           | The number of years between the student's current and previous jobs.             |
| `training_hours`         | The number of hours of training completed.                                       |
| `job_change`             | An indicator of whether the student is looking for a new job (`1`) or not (`0`). |

In [1]:
# Import necessary libraries
import pandas as pd

# Load the dataset
ds_jobs = pd.read_csv("customer_train.csv")

# View the dataset
ds_jobs.head()

student_id      city  ...  training_hours job_change
0        8949  city_103  ...              36        1.0
1       29725   city_40  ...              47        0.0
2       11561   city_21  ...              83        0.0
3       33241  city_115  ...              52        1.0
4         666  city_162  ...               8        0.0

[5 rows x 14 columns]

In [2]:

ds_jobs.columns
ds_jobs_transformed = ds_jobs.copy()


In [3]:
ds_jobs_transformed['relevant_experience'] = (
    ds_jobs_transformed['relevant_experience'] == 'Has relevant experience'
)

In [4]:
# You can verify this with:
ds_jobs_transformed.dtypes

student_id                  int64
city                       object
city_development_index    float64
gender                     object
relevant_experience          bool
enrolled_university        object
education_level            object
major_discipline           object
experience                 object
company_size               object
company_type               object
last_new_job               object
training_hours              int64
job_change                float64
dtype: object

In [5]:

# Example fix: Map any unexpected values to np.nan before conversion

import numpy as np

order = ['never', '1', '2', '3', '4', '>4']

# Set values not in 'order' to np.nan
ds_jobs_transformed.loc[
    ~ds_jobs_transformed['last_new_job'].isin(order), 'last_new_job'
] = np.nan

ds_jobs_transformed['last_new_job'] = pd.Categorical(
    ds_jobs_transformed['last_new_job'],
    categories=order,
    ordered=True
)

ds_jobs_transformed['last_new_job'].head()

0        1
1       >4
2    never
3    never
4        4
Name: last_new_job, dtype: category
Categories (6, object): ['never' < '1' < '2' < '3' < '4' < '>4']

In [6]:
# First, replace '>20' and '<1' with numeric values
ds_jobs_transformed['experience'] = ds_jobs_transformed['experience'].replace({
    '>20': 21,
    '<1': 0
})

# Now, convert all values to numeric
ds_jobs_transformed['experience'] = pd.to_numeric(ds_jobs_transformed['experience'], errors='coerce').astype('float16')

ds_jobs_transformed['experience'].head()

0    21.0
1    15.0
2     5.0
3     0.0
4    21.0
Name: experience, dtype: float16

In [7]:
import pandas as pd

ds_jobs_transformed = pd.read_csv("customer_train.csv")

ds_jobs_transformed['experience'] = ds_jobs_transformed['experience'].replace({
    '>20': '21',
    '<1': '0'
})

ds_jobs_transformed['experience'] = pd.to_numeric(
    ds_jobs_transformed['experience'],
    errors='coerce'
)

ds_jobs_transformed = ds_jobs_transformed.dropna(subset=['experience'])


ds_jobs_transformed = ds_jobs_transformed[
    (ds_jobs_transformed['experience'] >= 10) &
    (ds_jobs_transformed['company_size'].isin([
        '1000-4999',
        '5000-9999',
        '10000+'
    ]))
]
from pandas.api.types import CategoricalDtype

exp_order = [str(i) for i in range(0, 22)]
exp_type = CategoricalDtype(categories=exp_order, ordered=True)

ds_jobs_transformed['experience'] = (
    ds_jobs_transformed['experience']
    .astype(int)
    .astype(str)
    .astype(exp_type)
)
company_size_order = [
    '<10','10-49','50-99','100-499',
    '500-999','1000-4999','5000-9999','10000+'
]

ds_jobs_transformed['company_size'] = pd.Categorical(
    ds_jobs_transformed['company_size'],
    categories=company_size_order,
    ordered=True
)
ds_jobs_transformed.isna().sum()
ds_jobs_transformed.dtypes



student_id                   int64
city                        object
city_development_index     float64
gender                      object
relevant_experience         object
enrolled_university         object
education_level             object
major_discipline            object
experience                category
company_size              category
company_type                object
last_new_job                object
training_hours               int64
job_change                 float64
dtype: object

In [8]:

ds_jobs_transformed['job_change'] = (
    ds_jobs_transformed['job_change']
    .fillna(0)
    .astype(int)
    .map({0: False, 1: True})
)

ds_jobs_transformed['student_id'] = ds_jobs_transformed['student_id'].astype('int32')

ds_jobs_transformed['training_hours'] = ds_jobs_transformed['training_hours'].astype('int32')

ds_jobs_transformed['relevant_experience'] = (
    ds_jobs_transformed['relevant_experience']
    .map({
        'No relevant experience': False,
        'Has relevant experience': True
    })
)
ds_jobs_transformed['relevant_experience'].dtype


dtype('bool')

In [9]:
edu_order = ['no_enrollment', 'Part time course', 'Full time course']

ds_jobs_transformed['enrolled_university'] = pd.Categorical(
    ds_jobs_transformed['enrolled_university'],
    categories=edu_order,
    ordered=True
)
ds_jobs_transformed['enrolled_university'].dtype


CategoricalDtype(categories=['no_enrollment', 'Part time course', 'Full time course'], ordered=True)

In [10]:
ds_jobs_transformed['relevant_experience'].unique()
ds_jobs_transformed['relevant_experience'] = ds_jobs_transformed['relevant_experience'].map({
    'No relevant experience': False,
    'Has relevant experience': True
}).astype('bool')
ds_jobs_transformed['relevant_experience'].dtype


dtype('bool')

In [11]:
# Fix the job_change column: convert 0/1 or float 0.0/1.0 to boolean
ds_jobs_transformed['job_change'] = ds_jobs_transformed['job_change'].map({
    0: False,
    1: True,
    0.0: False,
    1.0: True
}).astype('bool')

# Fix the student_id column: ensure int32 type
ds_jobs_transformed['student_id'] = ds_jobs_transformed['student_id'].astype('int32')

# Fix the training_hours column: ensure int32 type
ds_jobs_transformed['training_hours'] = ds_jobs_transformed['training_hours'].astype('int32')

# Fix the city_development_index column: ensure float16 type
ds_jobs_transformed['city_development_index'] = ds_jobs_transformed['city_development_index'].astype('float16')

# Fix the city column: ensure category type
ds_jobs_transformed['city'] = ds_jobs_transformed['city'].astype('category')

# Fix the gender column: ensure category type
ds_jobs_transformed['gender'] = ds_jobs_transformed['gender'].astype('category')

# Fix the major_discipline column: ensure category type
ds_jobs_transformed['major_discipline'] = ds_jobs_transformed['major_discipline'].astype('category')

#Fix the company_type column
ds_jobs_transformed['company_type'] = ds_jobs_transformed['company_type'].astype('category')

#Fix the entrolled_university column
ds_jobs_transformed['enrolled_university'] = ds_jobs_transformed['enrolled_university'].astype('category')

edu_level_order = [
    'Primary School',
    'High School',
    'Graduate',
    'Masters',
    'Phd'
]

ds_jobs_transformed['education_level'] = pd.Categorical(
    ds_jobs_transformed['education_level'],
    categories=edu_level_order,
    ordered=True
)


company_size_order = [
    '<10',
    '10-49',
    '50-99',
    '100-499',
    '500-999',
    '1000-4999',
    '5000-9999',
    '10000'
]

ds_jobs_transformed['company_size'] = pd.Categorical(
    ds_jobs_transformed['company_size'],
    categories=company_size_order,
    ordered=True
)

# Check dtypes
ds_jobs_transformed[['job_change', 'student_id', 'training_hours', 'city_development_index', 'city', 'gender', 'major_discipline', 'company_type', 'enrolled_university','education_level','company_size']].dtypes

job_change                    bool
student_id                   int32
training_hours               int32
city_development_index     float16
city                      category
gender                    category
major_discipline          category
company_type              category
enrolled_university       category
education_level           category
company_size              category
dtype: object

In [12]:
last_job_order = ['never', '1', '2', '3', '4', '>4']

ds_jobs_transformed['last_new_job'] = pd.Categorical(
    ds_jobs_transformed['last_new_job'],
    categories=last_job_order,
    ordered=True
)

ds_jobs_transformed['last_new_job'].dtype


CategoricalDtype(categories=['never', '1', '2', '3', '4', '>4'], ordered=True)

In [13]:
ds_jobs_transformed.dtypes

student_id                   int32
city                      category
city_development_index     float16
gender                    category
relevant_experience           bool
enrolled_university       category
education_level           category
major_discipline          category
experience                category
company_size              category
company_type              category
last_new_job              category
training_hours               int32
job_change                    bool
dtype: object